In [1]:
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterator, *args, **kwargs):
        return iterator
    
import numpy as np
import scipy as sp
from scipy import constants
from pylab import *
import joblib
from numpy.linalg import inv
import iminuit

import os,sys
from importlib import reload
import copy
from collections import namedtuple

sys.path.append("../tracker")

In [2]:
import kalmanfilter as KF
import utilities as Util
import datatypes
import vertexfinder as VF

reload(VF)
reload(Util)

<module 'utilities' from '../tracker\\utilities.py'>

In [3]:
# -------------------------------------
# LS fit
# ------------------------------------

import numpy as np
class chi2_track_scattering:
    def __init__(self, hits):
        self.hits=hits
        self.func_code = iminuit.util.make_func_code(['x0', 'z0', 't0', 'Ax', 'Az', 'At'])
    def __call__(self, x0, z0, t0, Ax, Az, At):
        chi2_distance = 0

        cov_x, cov_z = self.get_cov(x0, z0, t0, Ax, Az, At)
        residuals = []

        for hit in self.hits:
            dy = (hit.y - self.hits[1].y)
            model_x = x0 + Ax*dy
            model_z = z0 + Az*dy
            model_t = t0 + At*dy
            residuals.append([model_x-hit.x, model_z-hit.z, model_t-hit.t])
        t_err = [hit.t_err for hit in hits]

        residuals=np.array(residuals)

        chi2_distance = residuals[:,0].T @ np.linalg.inv(cov_x) @residuals[:,0] +\
                        residuals[:,1].T @ np.linalg.inv(cov_z) @residuals[:,1] +\
                        np.sum([(residuals[:,2]/t_err)**2])
        return chi2_distance        

    def get_theta0(self, sin_theta, p=500):
        L_Al =  0.4
        L_Sc = 1.0 # [cm] Scintillator
        L_r_Al = 24.0111/2.7; # [cm] Radiation length Aluminum/ density of Aluminum
        L_r_Sc = 43; # [cm] Radiation length Scintillator (Saint-Gobain paper)

        L_rad = L_Al / L_r_Al + L_Sc / L_r_Sc; # [rad lengths] orthogonal to Layer
        L_rad /= sin_theta; # [rad lengths] in direction of track

        sigma_ms = 13.6 * np.sqrt(L_rad) * (1 + 0.038 * np.log(L_rad)); #
        sigma_ms /= p # [MeV] Divided by 1000 MeV

        return sigma_ms

    def get_cov(self, x0, z0, t0, Ax, Az, At):

        sin_theta = np.power(Ax**2+Az**2+1, -1/2)
        theta_0 = self.get_theta0(sin_theta)  
        hits = self.hits

        cov_xz = np.zeros((len(self.hits), len(self.hits)))
        for i in range(1,len(self.hits)):
            for j in range(1,len(self.hits)):
                cov_xz[i,j] = np.sum([(hits[i].y-hits[k].y)*(hits[j].y-hits[k].y)*theta_0**2 for k in range(0, min(i,j))])   

        cov_x = cov_xz* Ax**2
        cov_z = cov_xz* Az**2

        cov_x+=np.diag([hit.x_err**2 for hit in hits])
        cov_z+=np.diag([hit.z_err**2 for hit in hits])

        return np.array(cov_x), np.array(cov_z)


def fit_track_scattering(hits, guess):
    x0_init, z0_init,t0_init,Ax_init,Az_init,At_init = guess

    m = iminuit.Minuit(chi2_track_scattering(hits),x0=x0_init, z0=z0_init, t0=t0_init, Ax=Ax_init,Az=Az_init, At=At_init)
    m.limits["x0"]=(-100000,100000)
    m.limits["z0"]=(-100000,100000)
    m.limits["t0"]=(-100,1e5)
    m.limits["Ax"]=(-10,10) # Other
    m.limits["Az"]=(-10,10)
    m.limits["At"]=(0.001,0.2) # Beam direction; From MKS unit to cm/ns = 1e2/1e9=1e-7
    m.errors["x0"]=0.1
    m.errors["z0"]=0.1
    m.errors["t0"]=0.1
    m.errors["Ax"] = 0.0001
    m.errors["At"] = 0.0001
    m.errors["Az"] = 0.0001

    m.migrad()  # run optimiser
    m.hesse()   # run covariance estimator
    
    return m          

In [4]:
x = [241.62934581783023, 259.25, 221.49664808213277, 227.75]
y = [9894.0, 9975.599999999999, 10057.2, 10138.8]
z = [12456.75, 12450.5, 12461.25, 12450.5]
t = [40.91365706617539, 45.79668867914941, 48.189471653643565, 50.295569678622876]
layers = [0,1,2,3,]

hits = Util.hit.make_hits(x,y,z,t,layers)

In [5]:
# %timeit -n 1 fit_ls = fit_track_scattering(hits,guess)
# %timeit -n 1 Util.track.run_kf(hits, multiple_scattering=True)

In [6]:
guess = Util.track.guess_track(hits)
fit_ls = fit_track_scattering(hits,guess)
fit_ls.values, fit_ls.errors

(<ValueView x0=258.6952428118093 z0=12458.904134374345 t0=44.76956160118144 Ax=-0.18931185202159817 Az=0.026115162650500068 At=0.037412180180083576>,
 <ErrorView x0=1.2879471417618333 z0=0.9144510763217113 t0=0.3873106855588446 Ax=0.011298168582045243 Az=0.011154872561253804 At=0.0038742625161521486>)

In [7]:
kf = Util.track.run_kf(hits, multiple_scattering=True)
kf.Xsm[0], np.diag(np.sqrt(kf.Csm[0]))

c:\Users\Tom\.conda\envs\CV\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in sqrt
  


(array([ 2.58691818e+02,  1.24589018e+04,  4.47724563e+01, -1.88559159e-01,
         2.61473539e-02,  3.74197355e-02]),
 array([1.28411733, 0.95389681, 0.38742155, 0.01341403, 0.01164934,
        0.00387638]))